## Monte Carlo Markov Chain

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


# ------------------------------------------------------
#                  PARAMETERS (User Editable)
# ------------------------------------------------------
SEED = 123
N_SAMPLES = 30000       # total iterations
BURN_IN = 5000          # burn-in
SIGMA_PROPOSAL = 1.0    # proposal Normal(0, sigma^2)
X0 = 20                 # start far from target to show convergence

In [ ]:

# ------------------------------------------------------
#          1. Linear Congruential Generator (LCG)
# ------------------------------------------------------
def lcg(seed):
    """
    Simple LCG using Park-Miller values.
    Each call updates seed and returns a U(0,1) random number.
    """
    a = 16807
    m = 2**31 - 1
    seed = (a * seed) % m
    return seed, seed / m


# ------------------------------------------------------
#          2. Box-Muller to generate N(0, sigma^2)
# ------------------------------------------------------
def box_muller(seed, sigma=1.0):
    """
    Given an LCG seed, return:
    - updated seed
    - one Normal(0, sigma^2) random variable
    """
    seed, u1 = lcg(seed)
    seed, u2 = lcg(seed)

    # Avoid log(0)
    if u1 <= 0:
        u1 = 1e-12

    z = math.sqrt(-2 * math.log(u1)) * math.cos(2 * math.pi * u2)
    return seed, sigma * z

# ------------------------------------------------------
#         3. TARGET DISTRIBUTION (N(0,1) here)
# ------------------------------------------------------
def target_pdf(x):
    """
    Standard normal pdf. Easy to replace with your own target.
    """
    return math.e**(-np.abs(x))*(1 + 0.8*math.sin(3*x))


In [ ]:
# ------------------------------------------------------
#         4. Metropolis–Hastings MCMC
# ------------------------------------------------------
def mcmc(seed, n_samples, burn_in, sigma_prop, x0):
    samples = np.zeros(n_samples)
    x = x0
    accepted = 0

    for i in range(n_samples):
        # Generate proposal: x_new = x_old + Normal(0, sigma_prop^2)
        seed, noise = box_muller(seed, sigma=sigma_prop)
        proposal = x + noise

        # Compute acceptance ratio alpha
        t_old = target_pdf(x)
        t_new = target_pdf(proposal)

        # Avoid division by zero
        if t_old == 0:
            alpha = 1 if t_new > 0 else 0
        else:
            alpha = t_new / t_old

        # Draw U(0,1) from LCG
        seed, u = lcg(seed)

        # Accept or reject
        if u < min(1, alpha):
            x = proposal
            accepted += 1

        samples[i] = x

    acc_rate = accepted / n_samples
    return samples, acc_rate

In [ ]:
# ------------------------------------------------------
#                  RUN MCMC
# ------------------------------------------------------
samples, acc_rate = mcmc(SEED, N_SAMPLES, BURN_IN, SIGMA_PROPOSAL, X0)
post_samples = samples[BURN_IN:] # Applying burn in here 

print("Acceptance Rate =", acc_rate)
print("Post burn-in mean =", np.mean(post_samples))
print("Post burn-in std  =", np.std(post_samples))


# ------------------------------------------------------
#                  5. PLOTS
# ------------------------------------------------------
xs = np.linspace(-5, 5, 400)
pdf_vals = np.array([target_pdf(x) for x in xs])

# (i) Theoretical target (linear scale)
plt.figure(figsize=(7,4))
plt.plot(xs, pdf_vals)
plt.title("Theoretical Target PDF")
plt.xlabel("x")
plt.ylabel("pdf")
plt.grid()
plt.show()

# (i-b) Theoretical target (log scale)
plt.figure(figsize=(7,4))
plt.plot(xs, np.log(pdf_vals))
plt.title("Theoretical Target log(pdf)")
plt.xlabel("x")
plt.ylabel("log pdf")
plt.grid()
plt.show()

# (ii) MCMC histogram (linear)
plt.figure(figsize=(7,4))
plt.hist(post_samples, bins=60, density=True, alpha=0.6, label="MCMC Histogram")
plt.plot(xs, pdf_vals, 'r-', label="True PDF")
plt.title("MCMC vs True Distribution")
plt.xlabel("x")
plt.ylabel("Density")
plt.legend()
plt.grid()
plt.show()

# (ii-b) MCMC histogram (log scale)
plt.figure(figsize=(7,4))
plt.hist(post_samples, bins=60, density=True, alpha=0.6, log=True)
plt.scatter(xs, pdf_vals, s=8)
plt.title("MCMC Histogram (log scale)")
plt.xlabel("x")
plt.ylabel("log density")
plt.grid()
plt.show()

# (iii) Trace plot(linear scale)
plt.figure(figsize=(10,4))
plt.plot(samples, linewidth=0.6)
plt.axvline(BURN_IN, color='red', linestyle='--', label="Burn-in cutoff")
plt.title("Trace Plot(Linear Scale) of the MCMC Chain")
plt.xlabel("Iteration")
plt.ylabel("x value")
plt.legend()
plt.grid()
plt.show()


# Autocorrelation

In [ ]:
# ------------------------------------------------------
#         6. DIAGNOSTICS: ACF & ESS
# ------------------------------------------------------
from basic_operations import mean

def compute_acf(samples, K_max):
    """
    Implements ACF Pseudocode.
    Input: samples[] (post burn-in), K_max (lag limit)
    """
    M = len(samples)
    mean_x = mean(samples)
    rho = np.zeros(K_max + 1)
    
    # Pre-compute the denominator (variance * M essentially)
    # Pseudocode: sum( (samples[t] - mean_x)^2 )
    denominator = sum((samples - mean_x)**2)
    
    # Calculate rho for each lag k
    for k in range(K_max + 1):
        # Numerator computation
        # Pseudocode: sum( (samples[t] - mean_x) * (samples[t+k] - mean_x) )
        # We use numpy slicing to implement the 'for t' loop efficiently:
        # samples[:M-k] corresponds to samples[t]
        # samples[k:]   corresponds to samples[t+k]
        
        term1 = samples[:M-k] - mean_x
        term2 = samples[k:]   - mean_x
        numerator = sum(term1 * term2)
        
        rho[k] = numerator / denominator
        
    return rho

def compute_ess(rho, M):
    """
    Implements ESS Pseudocode.
    Input: rho (ACF array), M (number of samples)
    """
    # Pseudocode: tau_int = 1
    tau_int = 1.0
    
    # Pseudocode: for k from 1 to K_cutoff
    # We use the full length of the calculated rho as K_cutoff here.
    # Note: In practice, one often stops summing when rho becomes negative 
    # or noisy, but we will strictly follow the provided loop structure.
    K_cutoff = len(rho) - 1
    
    for k in range(1, K_cutoff + 1):
        tau_int += 2 * rho[k]
        
    # Pseudocode: ESS = M / tau_int
    ess = M / tau_int
    
    # Pseudocode: ESS_ratio = ESS / M
    ess_ratio = ess / M
    
    return ess, ess_ratio, tau_int

# --- CONFIGURATION FOR ACF ---
# Choose K_max so ACF visibly decays (e.g., 200-300 lags)
K_MAX = 200 

# --- EXECUTION ---
rho_values = compute_acf(post_samples, K_MAX)
ess_val, ess_r, tau_val = compute_ess(rho_values, len(post_samples))

# --- OUTPUT RESULTS ---
print("-" * 30)
print(f"Diagnostics (K_max={K_MAX})")
print("-" * 30)
print(f"Integrated Autocorrelation Time (tau_int): {tau_val:.4f}")
print(f"Effective Sample Size (ESS): {ess_val:.4f}")
print(f"ESS Ratio (ESS/N): {ess_r:.4f}")

# --- PLOTTING ACF (Matching the style of your screenshot) ---
plt.figure(figsize=(10, 5))

# Plot 1: Linear Scale (First 200 lags)
plt.subplot(1, 2, 1)
plt.plot(range(len(rho_values)), rho_values, '.-', linewidth=1, markersize=3)
plt.axhline(0, color='black', linewidth=0.5, alpha=0.5)
plt.title("Autocorrelation Function")
plt.xlabel("Lag k")
plt.ylabel("rho(k)")
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Plot 2: Log Scale (Optional, helps visualize decay rate)
plt.subplot(1, 2, 2)
# We only plot positive rho for log scale to avoid errors
positive_indices = rho_values > 0
plt.plot(np.arange(len(rho_values))[positive_indices], 
         rho_values[positive_indices], '.-', linewidth=1, markersize=3)
plt.yscale('log')
plt.title("Autocorrelation Function (Log Scale)")
plt.xlabel("Lag k")
plt.ylabel("rho(k)")
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
compute_acf(np.array([3,7,5,6,2]), 2)